# Calculating REE concentrations in zircon
This notebook allows you to calculate the basic concentration of REE in Zircon following Blereau et all 2020. 

Inputs:
There are currently no inputs to this notebook, though you may wish to ammend the equilibrium concentrations of REEs that are used, as well as the location, size and resolution of spot analysis





# DO NOT change this section
Please just run the cells as they are. To run code cells, select and press shift + enter

## Code imports
Let's start with code imports. To run code cells, select and press shift + enter

In [1]:
import math
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import scipy.special as erf
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import FileLinks, FileLink, DisplayObject



Bad key "text.kerning_factor" on line 4 in
/home/ellie/anaconda3/envs/py3/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Function definitions

In [2]:
def diffusion_profile(r, t, a, C0, D, siy):
    """diffusion profile for REE in zircon for instantaneous 
    spherical source diffuse throughout the surrounding medium

    inputs are:
        r= radius of diffusion
        t= time in years (1 yr = 31,556,926 s = siy)
        C0= uniform concentration in initial sphere (initial)
        a = radius of sphere
        D=
    outputs are:
        C
        C_change
    """
    #instantaneous spherical source diffuse throughout the surrounding medium   
    C = C0*((0.5) *  (erf.erf((a+r)/(2*(np.sqrt(D*t*siy)))) + erf.erf((a-r)/(2*(np.sqrt(D*t*siy)))))  - 1/r *np.sqrt((D*t*siy)/np.pi) * (np.exp(-(a-r)**2/(4*D*t*siy))-np.exp(-(a+r)**2/(4*D*t*siy)))) # not normalised
    
    # concentration change from initial value
    C_change = C / C0
    
    return [C, C_change]

def sectorarea(r, h1, h2=[]):
    """ returns the area of a segment for circle of radius r, 
    where h1 is the distance from centre to inner bounding chord
          h2 is the distance from centre to outer bounding chord
    if h2 is defined, the area between two chords at radius h1 and h2 wil be given
    """
    if h2 == []:
        h2 = r
    area_inner = r * (r * np.arccos( h1 / r) - h1 * np.sqrt( 1 - (h1**2 / r**2)))
    area_outer = r * (r * np.arccos( h2 / r) - h2 * np.sqrt( 1 - (h2**2 / r**2)))
    
    return area_inner - area_outer 

## insert a description here about what these tables are

The below tables are the values based on the equation within Cherniak et al., 1997

In [3]:
elements = ["Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu"]
temps = [850, 900, 950, 1000, 1050, 1100]
T_850_pd= pd.DataFrame([[850, 4117660.35, 780.17, 40.63, 2.12259093572083e-30, 1.97, 19.66],
                           [850,818932.3, 755.28, 36.4, 6.07207381840696e-30, 1.45, 3.63],
                           [850, 228722.09, 734.1, 34.73, 1.63814577570689e-29, 26.7, 24.27],
                           [850,95249.56, 717.85, 35.45, 3.88704535708724e-29, 12.23, 5.56],
                           [850,55017.45, 705.74, 38.01, 8.21411248096959e-29, 54.03, 15.89],
                           [850,41250.2, 697.04, 41.93, 1.56367397006138e-28, 12.1, 2.42],
                           [850,37797.49, 691.09, 46.74, 2.70953628982112e-28, 110.5, 16.25],
                           [850,40091.48, 687.3, 52.05, 4.31393656099103e-28, 21.87, 2.43]],
                          columns=["T_celcius",
                            "initial_diffusion_coefficient", 
                            "activation_energy", 
                            "activation_energy_error",
                            "diffusion_coefficient", 
                            "concentration", 
                            "Equilib_conc"], 
                        index=elements)

T_900_pd = pd.DataFrame([[900, 4117660.35, 780.17, 40.63, 7.47584307132256E-29, 1.97, 19.66],
                [900, 818932.3, 755.28, 36.4, 1.90886714212669E-28, 1.45, 3.63],
                [900, 228722.09, 734.1, 34.73, 4.67527055217335E-28, 26.7, 24.27],
                [900, 95249.56, 717.85, 35.45, 1.03005642325101E-27, 12.23, 5.56],
                [900, 55017.45, 705.74, 38.01, 2.05963762865096E-27, 54.03, 15.89],
                [900, 41250.2, 697.04, 41.93, 3.76814360871411E-27, 12.1, 2.42],
                [900, 37797.49, 691.09, 46.74, 6.35449675860049E-27, 110.5, 16.25],
                [900, 40091.48, 687.3, 52.05, 9.94354409048236E-27, 21.87, 2.43]],
                   columns=["T_celcius",
                            "initial_diffusion_coefficient", 
                            "activation_energy", 
                            "activation_energy_error",
                            "diffusion_coefficient", 
                            "concentration", 
                            "Equilib_conc"], 
                        index=elements)


T_950_pd =pd.DataFrame([[950, 4117660.35, 780.17, 40.63, 1.96778963894181E-27, 1.97, 19.66],
                [950, 818932.3, 755.28, 36.4, 4.52662554383351E-27, 1.45, 3.63],
                [950, 228722.09, 734.1, 34.73, 1.01450470367865E-26, 26.7, 24.27],
                [950, 95249.56, 717.85, 35.45, 2.08799499018551E-26, 12.23, 5.56],
                [950, 55017.45, 705.74, 38.01, 3.96836240938822E-26, 54.03, 15.89],
                [950, 41250.2, 697.04, 41.93, 7.00018312872847E-26, 12.1, 2.42],
                [950, 37797.49, 691.09, 46.74, 1.15141664932176E-25, 110.5, 16.25],
                [950, 40091.48, 687.3, 52.05, 1.77332600904736E-25, 21.87, 2.43]],
                     columns=["T_celcius",
                            "initial_diffusion_coefficient", 
                            "activation_energy", 
                            "activation_energy_error",
                            "diffusion_coefficient", 
                            "concentration", 
                            "Equilib_conc"], 
                        index=elements)



T_1000_pd =pd.DataFrame([[1000, 4117660.35, 780.17, 40.63, 4.00612612520478E-26, 1.97, 19.66],
                [1000, 818932.3, 755.28, 36.4, 8.37068419709278E-26, 1.45, 3.63],
                [1000, 228722.09, 734.1, 34.73, 1.72869073173745E-25, 26.7, 24.27],
                [1000, 95249.56, 717.85, 35.45, 3.34146847823377E-25, 12.23, 5.56],
                [1000, 55017.45, 705.74, 38.01, 6.06042779384192E-25, 54.03, 15.89],
                [1000, 41250.2, 697.04, 41.93, 1.03372969732327E-24, 12.1, 2.42],
                [1000, 37797.49, 691.09, 46.74, 1.66169103872795E-24, 110.5, 16.25],
                [1000, 40091.48, 687.3, 52.05, 2.52199796700586E-24, 21.87, 2.43]],
                     columns=["T_celcius",
                            "initial_diffusion_coefficient", 
                            "activation_energy", 
                            "activation_energy_error",
                            "diffusion_coefficient", 
                            "concentration", 
                            "Equilib_conc"], 
                        index=elements)


T_1050_pd = pd.DataFrame([[1050, 4117660.35, 780.17, 40.63, 6.49453363594283E-25, 1.97, 19.66],
                [1050, 818932.3, 755.28, 36.4, 1.24159849734451E-24, 1.45, 3.63],
                [1050, 228722.09, 734.1, 34.73, 2.37738446554115E-24, 26.7, 24.27],
                [1050, 95249.56, 717.85, 35.45, 4.33634950190383E-24, 12.23, 5.56],
                [1050, 55017.45, 705.74, 38.01, 7.53199159886072E-24, 54.03, 15.89],
                [1050, 41250.2, 697.04, 41.93, 1.24543854462706E-23, 12.1, 2.42],
                [1050, 37797.49, 691.09, 46.74, 1.95992792601782E-23, 110.5, 16.25],
                [1050, 40091.48, 687.3, 52.05, 2.93463288126692E-23, 21.87, 2.43]],
                       columns=["T_celcius",
                            "initial_diffusion_coefficient", 
                            "activation_energy", 
                            "activation_energy_error",
                            "diffusion_coefficient", 
                            "concentration", 
                            "Equilib_conc"], 
                        index=elements)


T_1100_pd = pd.DataFrame([[1100, 4117660.35, 780.17, 40.63, 8.59520030516088E-24, 1.97, 19.66],
                [1100, 818932.3, 755.28, 36.4, 1.51320465469538E-23, 1.45, 3.63],
                [1100, 228722.09, 734.1, 34.73, 2.70127943074511E-23, 26.7, 24.27],
                [1100, 95249.56, 717.85, 35.45, 4.669111120814E-23, 12.23, 5.56],
                [1100, 55017.45, 705.74, 38.01, 7.79125159346986E-23, 54.03, 15.89],
                [1100, 41250.2, 697.04, 41.93, 1.25173120183672E-22, 12.1, 2.42],
                [1100, 37797.49, 691.09, 46.74, 1.93141372017147E-22, 110.5, 16.25],
                [1100, 40091.48, 687.3, 52.05, 2.8558574788732E-22, 21.87, 2.43]],
                       columns=["T_celcius",
                            "initial_diffusion_coefficient", 
                            "activation_energy", 
                            "activation_energy_error",
                            "diffusion_coefficient", 
                            "concentration", 
                            "Equilib_conc"], 
                        index=elements)
tables = [T_850_pd, T_900_pd, T_950_pd, T_1000_pd, T_1050_pd, T_1100_pd]

In [4]:
for t in tables:
    temp = []
    for i in range(len(t)):
        temp.append(t['T_celcius'][i]+273.15)
    t['T_Kelvin']=temp
    
for t in tables:
    temp = []
    temp_p = []
    temp_n = []
    for i in range(len(t)):
        D0 = t['initial_diffusion_coefficient'][i]
        Ea = t['activation_energy'][i]
        Ea_err = t['activation_energy_error'][i]
        T = t['T_Kelvin'][i]
        temp.append(D0 * np.exp(-Ea/( 0.00831447 * T)))
        temp_p.append(D0 * np.exp(-(Ea-Ea_err)/( 0.00831447 * T)))
        temp_n.append(D0 * np.exp(-(Ea+Ea_err)/( 0.00831447 * T)))
    t['diffusion_coefficient'] = temp
    t['diffusion_coefficient_max'] = temp_p
    t['diffusion_coefficient_min'] = temp_n
    

## variables - you may change these
C0: uniform concentration in sphere (initial)

C: concentration at radius (r) at a time (t)

D: Diffusion coefficient (middle and maximum) m^2s

t: time in years (1 yr = 31,556,926 s = siy)

siy: 31556926

r: radius of interest in metres

a: radius of sphere

C_eq: equilibrium concentration


In [5]:
# time scales chosen are 0.5, 1, 5, 10, 30, 40, 100, 150, 200 My
times = [500000, 1000000, 5000000, 10000000, 30000000, 40000000, 100000000, 150000000, 200000000]

# conversion factor of years to seconds - 31556926 seconds in a year 
siy = 365.2425 * 24*60*60

# radius chosen as grain boundary is 50 microns (given in metres: 50 microns/10000=0.005 cm 0.00005 m)
a = 0.00005

# radii under investigation up to rim is 5-50 microns
radii = np.arange(0.0000005,0.00005, 0.0000005) 

Co = [21.626,5.082,50.967,17.792,69.916,14.52,126.75,24.3]

C_eq = [19.66, 3.63, 24.27, 5.56, 15.89, 2.42, 16.25, 2.43]

Normalised_Co = [110, 140, 210, 320, 440, 600, 780, 1000]
Normalised_C_eq = [110, 110, 110, 110, 110, 110, 110, 110]
# from Anders and Grevesse
Norm_values = [0.1966, 0.0363, 0.2427, 0.0556, 0.1589, 0.0242, 0.1625, 0.0243]

## Run function on all timesteps / radii

In [6]:
# this makes empty tables to store processed data.
T_850_conc = np.zeros((len(radii), len(times), len(elements)))
T_900_conc = np.zeros((len(radii), len(times), len(elements)))
T_950_conc = np.zeros((len(radii), len(times), len(elements)))
T_1000_conc = np.zeros((len(radii), len(times), len(elements)))
T_1050_conc = np.zeros((len(radii), len(times), len(elements)))
T_1100_conc = np.zeros((len(radii), len(times), len(elements)))
T_850_conc_ch = np.zeros((len(radii), len(times), len(elements)))
T_900_conc_ch = np.zeros((len(radii), len(times), len(elements)))
T_950_conc_ch = np.zeros((len(radii), len(times), len(elements)))
T_1000_conc_ch = np.zeros((len(radii), len(times), len(elements)))
T_1050_conc_ch = np.zeros((len(radii), len(times), len(elements)))
T_1100_conc_ch = np.zeros((len(radii), len(times), len(elements)))

# this makes a table of tables
tables_conc = [T_850_conc, T_900_conc, T_950_conc, T_1000_conc, T_1050_conc, T_1100_conc]
tables_conc_ch = [T_850_conc_ch, T_900_conc_ch, T_950_conc_ch, T_1000_conc_ch, T_1050_conc_ch, T_1100_conc_ch]

for t in range(len(tables)):
    D = tables[t]['diffusion_coefficient']
    Dp = tables[t]['diffusion_coefficient_max']
    Dn = tables[t]['diffusion_coefficient_min']
    C0 = tables[t]['concentration']
    temp_conc = tables_conc[t]
    temp_change = tables_conc_ch[t]
    for i in range(len(radii)):
        for j in range(len(times)):
            conc = diffusion_profile(radii[i], times[j], a, C0, D, siy)
            temp_conc[i][j][:] = conc[0]
            temp_change[i][j][:] = conc[1]


## Normalise wrt Chrondrite equilibrium values

In [7]:
# this makes empty tables to store processed data.
T_850_conc_eq = np.zeros((len(radii), len(times), len(elements)))
T_900_conc_eq = np.zeros((len(radii), len(times), len(elements)))
T_950_conc_eq = np.zeros((len(radii), len(times), len(elements)))
T_1000_conc_eq = np.zeros((len(radii), len(times), len(elements)))
T_1050_conc_eq = np.zeros((len(radii), len(times), len(elements)))
T_1100_conc_eq = np.zeros((len(radii), len(times), len(elements)))

# this makes a table of tables
tables_conc_eq = [T_850_conc_eq, T_900_conc_eq, T_950_conc_eq, T_1000_conc_eq, T_1050_conc_eq, T_1100_conc_eq]

for t in range(len(tables)):
    temp_conc_eq = tables_conc_eq[t]
    temp_conc =  tables_conc[t]
    for i in range(len(radii)):
        for j in range(len(times)):
            temp_conc_eq[i][j][:] = temp_conc[i][j][:] / Norm_values


In [8]:
# print(np.abs(radii - 40e-6).argmin())
tables_conc_eq[2][79, :,0]

array([10.02034588, 10.02034588, 10.02034588, 10.02034588, 10.02034429,
       10.02029529,  9.99001612,  9.88413757,  9.72250446])

# calculate theoretical analytical spot concentrations

calculate the concentration of a spot size
for example a 20 micron spot has a 10 micron radius and we want a 1 micron resolution
table should be an n-d array of element concentrations with distance

r = spot radius = 10

d = distance from centre of grain to place centre of spot. Default assumes a 50 micron grain, with spot entred at 40 microns (10 microns from rim)

res = resolution = 1
    

In [12]:
d = 40 
r = 10 
res = 1

In [15]:
def spot_fun(d, r, res, t, e, conc_table, normalise=True):
    
    temp_table = np.zeros((len(t), len(e)))
    
    # number of area segments to get desired resolution
    segs = int(r * 2 / res)
    # what radial distances 
    dist = [(d - r + i) for i in range(1, segs+1) ]

    dist_micron = [(d - r + i) *(1e-6) for i in range(1, segs+1) ]
    spot_seg_radii_idx = []

    for i in dist_micron:
        spot_seg_radii_idx.append((np.abs(i - radii)).argmin())


    norm_area = np.pi * r**2

    for j in range(len(t)):
        temp_conc = conc_table[:, j, :]

        for i in range(len(e)):
            for k in range(len(dist)):
                r_conc = temp_conc[spot_seg_radii_idx[k]][i] 
                norm_seg_area = sectorarea(r, r-k-1, r-k) / norm_area
                temp_table[j][i] += (r_conc+C_eq[i]) * norm_seg_area
            if normalise:
                temp_table[j][i] = temp_table[j][i] / Norm_values[i]
                if temp_table[j][i] < Normalised_C_eq[i]:
                    temp_table[j][i] = Normalised_C_eq[i]
    return temp_table





T_850_spot = spot_fun(d, r, res, times, elements, tables_conc[0])
T_900_spot = spot_fun(d, r, res, times, elements, tables_conc[1])
T_950_spot = spot_fun(d, r, res, times, elements, tables_conc[2])
T_1000_spot = spot_fun(d, r, res, times, elements, tables_conc[3])
T_1050_spot = spot_fun(d, r, res, times, elements, tables_conc[4])
T_1100_spot = spot_fun(d, r, res, times, elements, tables_conc[5])
tables_spot = [T_850_spot, T_900_spot, T_950_spot, T_1000_spot, T_1050_spot, T_1100_spot]

T_850_spot_10 = spot_fun(10, r, res, times, elements, tables_conc[0])
T_900_spot_10 = spot_fun(10, r, res, times, elements, tables_conc[1])
T_950_spot_10 = spot_fun(10, r, res, times, elements, tables_conc[2])
T_1000_spot_10 = spot_fun(10, r, res, times, elements, tables_conc[3])
T_1050_spot_10 = spot_fun(10, r, res, times, elements, tables_conc[4])
T_1100_spot_10 = spot_fun(10, r, res, times, elements, tables_conc[5])
tables_spot_10 = [T_850_spot_10, T_900_spot_10, T_950_spot_10, T_1000_spot_10, T_1050_spot_10, T_1100_spot_10]

T_850_spot_eq = np.zeros((len(times), len(elements)))
T_900_spot_eq = np.zeros((len(times), len(elements)))
T_950_spot_eq = np.zeros((len(times), len(elements)))
T_1000_spot_eq = np.zeros((len(times), len(elements)))
T_1050_spot_eq = np.zeros((len(times), len(elements)))
T_1100_spot_eq = np.zeros((len(times), len(elements)))
tables_spot_eq = [T_850_spot_eq, T_900_spot_eq, T_950_spot_eq, T_1000_spot_eq, T_1050_spot_eq, T_1100_spot_eq]



In [16]:
elements = ["Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu"]
time_labels = ["0.5My", "1.0My", "5.0My", "10.0My", "30.0My", "40.0My", "100.0My", "150.0My", "200.0My"]

T_850_spot_pd = pd.DataFrame(T_850_spot.transpose(),
                         columns=time_labels, 
                         index=elements)
T_900_spot_pd = pd.DataFrame(T_900_spot.transpose(),
                         columns=time_labels, 
                         index=elements)
T_950_spot_pd = pd.DataFrame(T_950_spot.transpose(),
                         columns=time_labels, 
                         index=elements)
T_1000_spot_pd = pd.DataFrame(T_1000_spot.transpose(),
                         columns=time_labels, 
                         index=elements)
T_1050_spot_pd = pd.DataFrame(T_1050_spot.transpose(),
                         columns=time_labels, 
                         index=elements)
T_1100_spot_pd = pd.DataFrame(T_1100_spot.transpose(),
                         columns=time_labels, 
                         index=elements)

# output tables

In [17]:
print(f"These tables are the concentration of REEs expected at a radial distance {d} microns")
print(f"with a spot of radius {r}, for a given temperature in degrees")

These tables are the concentration of REEs expected at a radial distance 40 microns
with a spot of radius 10, for a given temperature in degrees


In [18]:
print(f"for temperature = {temps[0]}")
T_850_spot_pd 

for temperature = 850


,0.5My,1.0My,5.0My,10.0My,30.0My,40.0My,100.0My,150.0My,200.0My
Gd,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
Tb,139.571475,139.571441,139.571296,139.571187,139.570916,139.570816,139.570385,139.570099,139.569677
Dy,208.983753,208.983598,208.982942,208.982451,208.981223,208.980772,208.975113,208.955961,208.922664
Ho,317.906976,317.906497,317.904478,317.902965,317.898260,317.892182,317.723838,317.502808,317.276921
Er,436.844526,436.843451,436.838913,436.835441,436.759401,436.655894,435.794104,435.102094,434.462323
Tm,595.321475,595.319294,595.310025,595.290051,594.706347,594.293195,591.996587,590.341230,588.831994
Yb,773.634940,773.631035,773.606807,773.414876,771.568912,770.641387,765.888114,762.531620,759.479569
Lu,991.572392,991.565872,991.430846,990.645607,986.707685,984.951392,976.114395,969.904114,964.257619


In [19]:
print(f"for temperature = {temps[1]}")
T_900_spot_pd 

for temperature = 900


,0.5My,1.0My,5.0My,10.0My,30.0My,40.0My,100.0My,150.0My,200.0My
Gd,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
Tb,139.571093,139.570901,139.570054,139.566658,139.500288,139.459987,139.245571,139.092654,138.953442
Dy,208.982126,208.981296,208.959818,208.848666,208.332284,208.104396,206.959587,206.155314,205.424010
Ho,317.902187,317.899368,317.582688,316.995488,315.082604,314.272550,310.225497,307.382417,304.797161
Er,436.833581,436.795255,435.437154,433.856740,428.960623,426.897846,416.587134,409.357538,402.821026
Tm,595.267653,594.937316,591.299851,587.678203,576.548881,571.858815,548.479841,532.327790,518.080314
Yb,773.293907,772.204614,764.692435,757.508746,735.443486,726.152206,680.475852,650.295339,624.953120
Lu,990.342430,988.016364,974.155747,961.029510,920.727978,903.847325,823.694616,774.458509,735.204742


In [20]:
print(f"for temperature = {temps[2]}")
T_950_spot_pd 

for temperature = 950


,0.5My,1.0My,5.0My,10.0My,30.0My,40.0My,100.0My,150.0My,200.0My
Gd,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
Tb,139.563547,139.525511,139.187069,138.856812,137.842399,137.414896,135.290001,133.837870,132.575310
Dy,208.826103,208.536614,206.817541,205.189883,200.192875,198.100751,188.189427,182.124415,177.299691
Ho,316.981217,315.948724,310.148674,304.670546,288.085478,281.435210,253.554217,238.760526,227.664730
Er,433.966283,431.410415,417.164622,403.762632,365.539165,351.726373,300.450197,275.322060,256.681620
Tm,588.160364,582.499814,550.997299,522.006198,448.611308,424.980999,342.499547,302.723894,273.401413
Yb,758.772652,747.752260,686.871932,634.066799,517.481610,482.723726,363.028114,306.475617,267.207608
Lu,963.693103,943.778052,836.207718,751.322543,582.887534,534.271125,368.468300,296.578802,251.224652


In [21]:
print(f"for temperature = {temps[3]}")
T_1000_spot_pd 

for temperature = 1000


,0.5My,1.0My,5.0My,10.0My,30.0My,40.0My,100.0My,150.0My,200.0My
Gd,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000,110.000000
Tb,138.903358,138.389152,135.531400,132.941964,126.693064,124.744122,117.996938,114.749885,112.388471
Dy,205.643104,203.253384,190.319342,180.044833,159.545279,153.622261,133.393228,124.557094,118.947594
Ho,306.744406,299.072009,261.177175,236.328638,191.247037,178.396638,139.133119,125.864825,118.649234
Er,409.781026,391.849535,316.475618,274.221451,199.718899,180.266164,132.315034,119.822755,113.701925
Tm,536.330016,499.697071,370.962238,304.363026,195.379082,172.115019,124.768775,114.519524,110.000000
Yb,661.370422,598.850447,407.376861,311.951363,180.460135,158.042694,117.979963,110.263205,110.000000
Lu,795.961485,700.838939,431.609001,305.709530,165.828767,146.012213,113.366179,110.000000,110.000000


In [22]:
print(f"for temperature = {temps[4]}")
T_1050_spot_pd 

for temperature = 1050


,0.5My,1.0My,5.0My,10.0My,30.0My,40.0My,100.0My,150.0My,200.0My
Gd,110.000000,110.000000,110.000000,110.000000,110.0,110.0,110.0,110.0,110.0
Tb,134.196097,130.974862,120.290981,114.846546,110.0,110.0,110.0,110.0,110.0
Dy,185.952147,174.515751,141.863496,126.392634,110.0,110.0,110.0,110.0,110.0
Ho,252.274069,226.230520,156.734611,130.211549,110.0,110.0,110.0,110.0,110.0
Er,303.675745,260.209420,153.917428,125.014006,110.0,110.0,110.0,110.0,110.0
Tm,353.705684,285.531813,146.088469,119.460473,110.0,110.0,110.0,110.0,110.0
Yb,385.284824,289.158880,136.087535,114.354023,110.0,110.0,110.0,110.0,110.0
Lu,403.706000,280.607321,128.181399,110.800883,110.0,110.0,110.0,110.0,110.0


In [23]:
print(f"for temperature = {temps[5]}")
T_1100_spot_pd 

for temperature = 1100


,0.5My,1.0My,5.0My,10.0My,30.0My,40.0My,100.0My,150.0My,200.0My
Gd,110.000000,110.000000,110.0,110.0,110.0,110.0,110.0,110.0,110.0
Tb,118.790302,113.228063,110.0,110.0,110.0,110.0,110.0,110.0,110.0
Dy,139.000379,123.740761,110.0,110.0,110.0,110.0,110.0,110.0,110.0
Ho,153.566719,127.959144,110.0,110.0,110.0,110.0,110.0,110.0,110.0
Er,152.155280,124.022501,110.0,110.0,110.0,110.0,110.0,110.0,110.0
Tm,145.847927,119.345262,110.0,110.0,110.0,110.0,110.0,110.0,110.0
Yb,136.789303,114.657529,110.0,110.0,110.0,110.0,110.0,110.0,110.0
Lu,129.247460,111.233605,110.0,110.0,110.0,110.0,110.0,110.0,110.0


# Have a play with plotting parameters

again, just use "shift" + "enter" on the keyboard to run the cell, then play with the interactive tool

The only option to change in the code, is if you would like to save the figures. remove the # symbol before those two lines to save in the figures.


In [32]:
%matplotlib inline

In [33]:
plt.close()
plt.rcParams['figure.figsize'] = [7, 7]

In [34]:
plt.close()
def plot_concs(Element, Temperature, Time_Scale): #C/Co v x distribution
    
    try:
        El_ix = elements.index(Element)
        e_range=1
    except ValueError:
        e_range=len(elements)
        legend_labels = elements
        
    try:      
        Tp_ix = temps.index(float(Temperature))
        T_range = 1
        
    except ValueError:
        T_range=len(tables)
        legend_labels = ["850", "900", "950", "1000", "1050", "1100"]
        
    try:
        TS_ix = time_labels.index(Time_Scale)
        t_range=1
    except ValueError:
        t_range = len(times)
        legend_labels = [i/1e6 for i in times]
    

    for T in range(T_range):
        if T_range==1:
            T = Tp_ix
#     temp_spot = tables_spot[t]

        for t in range(t_range):
            if t_range==1:
                t = TS_ix
            for e in range(e_range):
                if e_range==1:
                    e = El_ix
                temp_conc = tables_conc[T][:, t, e]
                #1000000 m to microns
                plt.plot(radii*1000000, temp_conc)


    plt.xlabel="Radius (microns)"
    plt.ylabel="C/C0 (ppm)"
    try:
        plt.legend(legend_labels)
    except UnboundLocalError:
        pass

#     outname = f"{Element}_{Temperature}_{Time_Scale}_normalised_spot_concentrations.png"  
#     plt.savefig(outname)#, dpi=None)
                
interact(plot_concs, Element=["Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "all"], Temperature= ["850", "900", "950", "1000", "1050", "1100", "all"], Time_Scale=["0.5My", "1.0My", "5.0My", "10.0My", "30.0My", "40.0My", "100.0My", "150.0My", "200.0My", "all"])

interactive(children=(Dropdown(description='Element', options=('Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu',…

<function __main__.plot_concs(Element, Temperature, Time_Scale)>

In [35]:
plt.close()
def plot_concs(Temperature, Time_Scale): #C/Co v x distribution  (Element, Temperature, Time_Scale)
    plt.figure()
    try:      
        Tp_ix = temps.index(float(Temperature))
        T_range = 1
        
    except ValueError:
        T_range=len(tables)
        legend_labels = ["850", "900", "950", "1000", "1050", "1100"]
        
    try:
        TS_ix = time_labels.index(Time_Scale)
        t_range=1
    except ValueError:
        t_range = len(times)
        legend_labels=([i/1e6 for i in times])
    

    for T in range(T_range):
        if T_range==1:
            T = Tp_ix
        for t in range(t_range):
            if t_range==1:
                t = TS_ix
            
            temp_spot = tables_spot[T][t, :]
            plt.plot(elements, temp_spot)

    
    try:
        plt.legend(legend_labels)
    except:
        pass
   
    plt.xticks(np.arange(len(elements)),elements)
    plt.plot(elements, Normalised_C_eq, color='k', label='C_eq')
    plt.plot(elements, Normalised_Co, color='grey', label='C_o')

    plt.ylabel="REE/Chondrite (ppm)"
    plt.yscale("log")
    
#     outname = f"{Temperature}_{Time_Scale}_normalised_spot_concentrations.png"  
#     plt.savefig(outname)#, dpi=None)
                
interact(plot_concs, Temperature= ["850", "900", "950", "1000", "1050", "1100", "all"], Time_Scale=["0.5My", "1.0My", "5.0My", "10.0My", "30.0My", "40.0My", "100.0My", "150.0My", "200.0My", "all"]) 

interactive(children=(Dropdown(description='Temperature', options=('850', '900', '950', '1000', '1050', '1100'…

<function __main__.plot_concs(Temperature, Time_Scale)>

In [36]:
plt.close()
def plot_concs(Temperature, Time_Scale): #C/Co v x distribution  (Element, Temperature, Time_Scale)
    plt.figure()
    try:      
        Tp_ix = temps.index(float(Temperature))
        T_range = 1
        
    except ValueError:
        T_range=len(tables)
        legend_labels = ["850", "900", "950", "1000", "1050", "1100"]
        
    try:
        TS_ix = time_labels.index(Time_Scale)
        t_range=1
    except ValueError:
        t_range = len(times)
        legend_labels=([i/1e6 for i in times])
    

    for T in range(T_range):
        if T_range==1:
            T = Tp_ix
        for t in range(t_range):
            if t_range==1:
                t = TS_ix
            
            temp_spot_outer = tables_spot[T][t, :]
            temp_spot_inner = tables_spot_10[T][t, :]
            plt.fill_between(elements, temp_spot_outer, temp_spot_inner)
#             plt.plot(elements, temp_spot)

    
    try:
        plt.legend(legend_labels)
    except:
        pass
   
    plt.xticks(np.arange(len(elements)),elements)
    plt.plot(elements, Normalised_C_eq, color='k', label='C_eq')
    plt.plot(elements, Normalised_Co, color='grey', label='C_o')

    plt.ylabel="REE/Chondrite (ppm)"
    plt.yscale("log")
    
#     outname = f"{Temperature}_{Time_Scale}_normalised_spot_concentrations.png"  
#     plt.savefig(outname)#, dpi=None)
                
interact(plot_concs, Temperature= ["850", "900", "950", "1000", "1050", "1100", "all"], Time_Scale=["0.5My", "1.0My", "5.0My", "10.0My", "30.0My", "40.0My", "100.0My", "150.0My", "200.0My", "all"]) 



interactive(children=(Dropdown(description='Temperature', options=('850', '900', '950', '1000', '1050', '1100'…

<function __main__.plot_concs(Temperature, Time_Scale)>